# Download do Dataset

In [1]:
#https://www.kaggle.com/datasets/blastchar/telco-customer-churn/data

In [2]:
#%%sh
#pip install kaggle

In [3]:
#!mkdir -p ~/.kaggle
#!mv kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

In [4]:
#!kaggle datasets download -d blastchar/telco-customer-churn

# Importação de Bibliotecas

In [5]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt

# Lê Dataset e Trata Variáveis

In [6]:
alvo = 'Churn'
df_tot = pd.read_csv('telco-customer-churn.zip', dtype = {'SeniorCitizen': str})
df_tot[alvo] = df_tot[alvo].map({'Yes': 1, 'No': 0})
df_tot.loc[df_tot['TotalCharges'] == ' ', 'TotalCharges'] = -1
df_tot['TotalCharges'] = df_tot['TotalCharges'].astype(float)

In [7]:
colunas_id = ['customerID']
features_categoricas = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
                        'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                        'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
features_numericas = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [8]:
dict_dummies = {}
for f in features_categoricas:
    lista_dummies = list(df_tot[f].value_counts().index[:-1])
    dict_dummies[f] = lista_dummies

for k,v in dict_dummies.items():
    for value in v:
        df_tot[k + '_' + value] = (df_tot[k] == value).astype(int)
    df_tot.drop(k, axis = 1, inplace = True)
df_tot.columns = [c.replace(' ', '_') for c in df_tot.columns]

In [9]:
features = ['tenure', 'MonthlyCharges', 'TotalCharges',
            'gender_Male', 'SeniorCitizen_0', 'Partner_No', 'Dependents_No',
            'PhoneService_Yes', 'PaperlessBilling_Yes', 'MultipleLines_No',
            'MultipleLines_Yes', 'InternetService_Fiber_optic',
            'InternetService_DSL', 'OnlineSecurity_No', 'OnlineSecurity_Yes',
            'OnlineBackup_No', 'OnlineBackup_Yes', 'DeviceProtection_No',
            'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_Yes',
            'StreamingTV_No', 'StreamingTV_Yes', 'StreamingMovies_No',
            'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_Two_year',
            'PaymentMethod_Electronic_check', 'PaymentMethod_Mailed_check',
            'PaymentMethod_Bank_transfer_(automatic)',
            'Churn']

In [10]:
X = StandardScaler().fit_transform(df_tot[features])

# Agrupamento

In [11]:
best_k, best_s = None, -1
for k in range(2, 11):
    km = KMeans(n_clusters = k, n_init = 10, random_state = 42).fit(X)
    s = silhouette_score(X, km.labels_)
    if s > best_s:
        best_k, best_s = k, s
print(f"Melhor k pelo silhouette: {best_k} (score = {best_s:.3f})")

Melhor k pelo silhouette: 4 (score = 0.174)


In [12]:
k = 4
km = KMeans(n_clusters = k, n_init = 10, random_state = 42)
labels = km.fit_predict(X)

df_clusters = df_tot.assign(cluster = labels)

# Anomalia

In [13]:
iso = IsolationForest(
    n_estimators = 300,
    max_samples = 'auto',
    contamination = 'auto',
    random_state = 42,
    n_jobs = -1
)
iso.fit(X)

anom_score = -iso.score_samples(X)
df_rank = df_clusters.assign(anom_score = anom_score)

# Analise

In [14]:
def f_agg(x):
    d = {}
    d['Perc_Base'] = x.shape[0]*100/df_tot.shape[0]
    d['Perc_Churn'] = x['Churn'].mean()*100
    d['Score_Anomalia'] = x['anom_score'].mean()
    return pd.Series(d)

display(df_rank.groupby('cluster').apply(lambda x: f_agg(x), include_groups = False).sort_values('Score_Anomalia').style.format(precision = 2))

,Perc_Base,Perc_Churn,Score_Anomalia
cluster,,,
2,21.67,7.40,0.49
0,29.04,56.38,0.54
3,23.07,25.72,0.56
1,26.22,10.02,0.57


In [15]:
indices_oculto = df_clusters[df_clusters['cluster'] == 3].index

# Split Conjunto Oculto

In [17]:
alvo = 'Churn'
df_tot = pd.read_csv('telco-customer-churn.zip', dtype = str)

display(df_tot['Churn'].value_counts()/len(df_tot))

print(50*'=')
######################

#df_oculto = df_tot[df_tot.index.isin(indices_oculto)]
#df = df_tot[~df_tot.index.isin(indices_oculto)]

df_oculto = df_tot.sample(frac = 0.5, random_state = 42)
df = df_tot[~df_tot.index.isin(df_oculto.index)]

print('')
print(df_oculto.shape[0])
display(df_oculto['Churn'].value_counts()/len(df_oculto))

print('')
print(df.shape[0])
display(df['Churn'].value_counts()/len(df))

######################

df.to_csv('telco-customer-churn-real.zip', index = False, compression = 'zip')
df_oculto.to_csv('telco-customer-churn-oculto.zip', index = False, compression = 'zip')

,count
Churn,
No,0.73463
Yes,0.26537



3522


,count
Churn,
No,0.731403
Yes,0.268597



3521


,count
Churn,
No,0.737859
Yes,0.262141
